In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pydicom

In [2]:
DATA_PATH = "..\\data CBIS-DDSM\\"
metadata_df = pd.read_csv(DATA_PATH + "metadata-full.csv")

In [3]:
metadata_df = metadata_df.drop(["Data Description URI","Series UID", "Subject ID", "Study UID", "SOP Class Name",
         "SOP Class UID", "Study Date", "Download Timestamp", "Collection", "3rd Party Analysis",
         "Study Description", "Manufacturer", "Modality"],axis=1)

In [4]:
metadata_df.head()

,Series Description,Number of Images,File Size,File Location
0,full mammogram images,1,27.84 MB,.\CBIS-DDSM\Calc-Test_P_00038_LEFT_CC\08-29-20...
1,full mammogram images,1,28.97 MB,.\CBIS-DDSM\Calc-Test_P_00038_LEFT_MLO\08-29-2...
2,ROI mask images,2,14.06 MB,.\CBIS-DDSM\Calc-Test_P_00038_LEFT_CC_1\08-29-...
3,ROI mask images,2,14.62 MB,.\CBIS-DDSM\Calc-Test_P_00038_LEFT_MLO_1\08-29...
4,full mammogram images,1,25.73 MB,.\CBIS-DDSM\Calc-Test_P_00038_RIGHT_CC\08-29-2...


In [5]:
LABELS_PATH = "../data CBIS-DDSM/Labels/"
mass_train, mass_test, calc_train, calc_test = pd.read_csv(LABELS_PATH + "mass_case_description_train_set.csv"), pd.read_csv(LABELS_PATH + "mass_case_description_test_set.csv"), pd.read_csv(LABELS_PATH + "calc_case_description_train_set.csv"),pd.read_csv(LABELS_PATH + "calc_case_description_test_set.csv")

In [6]:
calc_train = calc_train.drop(["breast density","left or right breast","image view","abnormality id","calc type","calc distribution","pathology","subtlety"],axis=1)
calc_test = calc_test.drop(["breast density","left or right breast","image view","abnormality id","calc type","calc distribution","pathology","subtlety"],axis=1)

mass_train = mass_train.drop(["breast_density","left or right breast","image view","abnormality id","mass shape","mass margins","pathology","subtlety"],axis=1)
mass_test = mass_test.drop(["breast_density","left or right breast","image view","abnormality id","mass shape","mass margins","pathology","subtlety"],axis=1)

In [7]:
for df in [mass_train, mass_test, calc_train, calc_test]:
    df["image file path"] = df["image file path"].apply(lambda x: str(x)[:str(x).index("/")+1] +"raw.dcm",1)
    df["cropped image file path"] = df["cropped image file path"].apply(lambda x: str(x)[:str(x).index("/")+1] +f"cropped{(str(x)[str(x).index('/')-1])}.dcm",1)
    df["ROI mask file path"] = df["ROI mask file path"].apply(lambda x: str(x)[:str(x).index("/")+1] +f"roi{(str(x)[str(x).index('/')-1])}.dcm",1)
    print(df.head(3))

  patient_id abnormality type  assessment  \
0    P_00001             mass           4   
1    P_00001             mass           4   
2    P_00004             mass           4   

                          image file path  \
0   Mass-Training_P_00001_LEFT_CC/raw.dcm   
1  Mass-Training_P_00001_LEFT_MLO/raw.dcm   
2   Mass-Training_P_00004_LEFT_CC/raw.dcm   

                         cropped image file path  \
0   Mass-Training_P_00001_LEFT_CC_1/cropped1.dcm   
1  Mass-Training_P_00001_LEFT_MLO_1/cropped1.dcm   
2   Mass-Training_P_00004_LEFT_CC_1/cropped1.dcm   

                          ROI mask file path  
0   Mass-Training_P_00001_LEFT_CC_1/roi1.dcm  
1  Mass-Training_P_00001_LEFT_MLO_1/roi1.dcm  
2   Mass-Training_P_00004_LEFT_CC_1/roi1.dcm  
  patient_id abnormality type  assessment                     image file path  \
0    P_00016             mass           5   Mass-Test_P_00016_LEFT_CC/raw.dcm   
1    P_00016             mass           5  Mass-Test_P_00016_LEFT_MLO/raw.dcm  

C:\Users\yusuf\AppData\Local\Temp\ipykernel_19956\1632652628.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df["image file path"] = df["image file path"].apply(lambda x: str(x)[:str(x).index("/")+1] +"raw.dcm",1)
C:\Users\yusuf\AppData\Local\Temp\ipykernel_19956\1632652628.py:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df["cropped image file path"] = df["cropped image file path"].apply(lambda x: str(x)[:str(x).index("/")+1] +f"cropped{(str(x)[str(x).index('/')-1])}.dcm",1)
C:\Users\yusuf\AppData\Local\Temp\ipykernel_19956\1632652628.py:4: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

In [9]:
calc_test["assessment"].value_counts()

assessment
4    178
2     71
5     40
3     24
0     13
Name: count, dtype: int64

In [29]:
calc_test.head()

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path
0,P_00038,calcification,4,Calc-Test_P_00038_LEFT_CC/raw.dcm,Calc-Test_P_00038_LEFT_CC_1/cropped1.dcm,Calc-Test_P_00038_LEFT_CC_1/roi1.dcm
1,P_00038,calcification,4,Calc-Test_P_00038_LEFT_MLO/raw.dcm,Calc-Test_P_00038_LEFT_MLO_1/cropped1.dcm,Calc-Test_P_00038_LEFT_MLO_1/roi1.dcm
2,P_00038,calcification,2,Calc-Test_P_00038_RIGHT_CC/raw.dcm,Calc-Test_P_00038_RIGHT_CC_1/cropped1.dcm,Calc-Test_P_00038_RIGHT_CC_1/roi1.dcm
3,P_00038,calcification,2,Calc-Test_P_00038_RIGHT_CC/raw.dcm,Calc-Test_P_00038_RIGHT_CC_2/cropped2.dcm,Calc-Test_P_00038_RIGHT_CC_2/roi2.dcm
4,P_00038,calcification,2,Calc-Test_P_00038_RIGHT_MLO/raw.dcm,Calc-Test_P_00038_RIGHT_MLO_1/cropped1.dcm,Calc-Test_P_00038_RIGHT_MLO_1/roi1.dcm


In [41]:
calc_df = calc_test.merge(calc_train, how="outer").sort_values("patient_id").reset_index().drop("index",axis=1)
calc_df["assessment"].value_counts()

assessment
4    931
2    553
5    199
3    113
0     76
Name: count, dtype: int64

In [55]:
sample_size = 100

# Create an empty dataframe to store the sampled data
sampled_calc_df = pd.DataFrame(columns=calc_df.columns)

sampled_dfs = []

# Iterate over each unique assessment value
for assessment_value in [0,2,3,4,5]:  # Range from 0 to 5
    # Select rows with the current assessment value
    subset = calc_df[calc_df['assessment'] == assessment_value]
    
    # Calculate the number of rows to sample for this assessment value
    num_samples = min(len(subset), sample_size // 5)  # Equal distribution
    
    # Sample the subset and append to the sampled dataframe
    sampled_dfs.append(subset.sample(n=num_samples))

sampled_calc_df = pd.concat(sampled_dfs)

In [56]:
sampled_calc_df

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path
1810,P_02368,calcification,0,Calc-Training_P_02368_RIGHT_CC/raw.dcm,Calc-Training_P_02368_RIGHT_CC_1/cropped1.dcm,Calc-Training_P_02368_RIGHT_CC_1/roi1.dcm
1822,P_02409,calcification,0,Calc-Training_P_02409_RIGHT_CC/raw.dcm,Calc-Training_P_02409_RIGHT_CC_1/cropped1.dcm,Calc-Training_P_02409_RIGHT_CC_1/roi1.dcm
1805,P_02323,calcification,0,Calc-Training_P_02323_LEFT_CC/raw.dcm,Calc-Training_P_02323_LEFT_CC_1/cropped1.dcm,Calc-Training_P_02323_LEFT_CC_1/roi1.dcm
1803,P_02318,calcification,0,Calc-Training_P_02318_RIGHT_MLO/raw.dcm,Calc-Training_P_02318_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_02318_RIGHT_MLO_1/roi1.dcm
1865,P_02563,calcification,0,Calc-Training_P_02563_RIGHT_MLO/raw.dcm,Calc-Training_P_02563_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_02563_RIGHT_MLO_1/roi1.dcm
...,...,...,...,...,...,...
934,P_01002,calcification,5,Calc-Training_P_01002_LEFT_MLO/raw.dcm,Calc-Training_P_01002_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_01002_LEFT_MLO_1/roi1.dcm
1260,P_01346,calcification,5,Calc-Training_P_01346_LEFT_CC/raw.dcm,Calc-Training_P_01346_LEFT_CC_1/cropped1.dcm,Calc-Training_P_01346_LEFT_CC_1/roi1.dcm
1609,P_01743,calcification,5,Calc-Test_P_01743_RIGHT_MLO/raw.dcm,Calc-Test_P_01743_RIGHT_MLO_1/cropped1.dcm,Calc-Test_P_01743_RIGHT_MLO_1/roi1.dcm
1337,P_01425,calcification,5,Calc-Test_P_01425_LEFT_CC/raw.dcm,Calc-Test_P_01425_LEFT_CC_1/cropped1.dcm,Calc-Test_P_01425_LEFT_CC_1/roi1.dcm


In [67]:
mass_df = mass_test.merge(mass_train.iloc[:mass_train.shape[0]//4,:], how="outer").sort_values("patient_id").reset_index().drop("index",axis=1)
mass_df["assessment"].value_counts()

assessment
4    308
3    174
5    137
0     48
2     38
1      2
Name: count, dtype: int64

In [68]:
sample_size = 100

# Create an empty dataframe to store the sampled data
sampled_mass_df = pd.DataFrame(columns=mass_df.columns)

sampled_dfs = []

# Iterate over each unique assessment value
for assessment_value in [0,2,3,4,5]:  # Range from 0 to 5
    # Select rows with the current assessment value
    subset = mass_df[mass_df['assessment'] == assessment_value]
    
    # Calculate the number of rows to sample for this assessment value
    num_samples = min(len(subset), sample_size // 5)  # Equal distribution
    
    # Sample the subset and append to the sampled dataframe
    sampled_dfs.append(subset.sample(n=num_samples))

sampled_mass_df = pd.concat(sampled_dfs)

In [69]:
sampled_mass_df

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path
106,P_00108,mass,0,Mass-Training_P_00108_LEFT_MLO/raw.dcm,Mass-Training_P_00108_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00108_LEFT_MLO_1/roi1.dcm
56,P_00065,mass,0,Mass-Training_P_00065_LEFT_CC/raw.dcm,Mass-Training_P_00065_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00065_LEFT_CC_1/roi1.dcm
432,P_00457,mass,0,Mass-Test_P_00457_LEFT_MLO/raw.dcm,Mass-Test_P_00457_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_00457_LEFT_MLO_1/roi1.dcm
46,P_00055,mass,0,Mass-Training_P_00055_LEFT_CC/raw.dcm,Mass-Training_P_00055_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00055_LEFT_CC_1/roi1.dcm
519,P_00922,mass,0,Mass-Test_P_00922_RIGHT_MLO/raw.dcm,Mass-Test_P_00922_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_00922_RIGHT_MLO_1/roi1.dcm
...,...,...,...,...,...,...
108,P_00110,mass,5,Mass-Training_P_00110_LEFT_MLO/raw.dcm,Mass-Training_P_00110_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00110_LEFT_MLO_1/roi1.dcm
577,P_01204,mass,5,Mass-Test_P_01204_RIGHT_MLO/raw.dcm,Mass-Test_P_01204_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_01204_RIGHT_MLO_1/roi1.dcm
545,P_00987,mass,5,Mass-Test_P_00987_LEFT_CC/raw.dcm,Mass-Test_P_00987_LEFT_CC_1/cropped1.dcm,Mass-Test_P_00987_LEFT_CC_1/roi1.dcm
565,P_01119,mass,5,Mass-Test_P_01119_LEFT_MLO/raw.dcm,Mass-Test_P_01119_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_01119_LEFT_MLO_1/roi1.dcm


In [70]:
sampled_mass_df["raw_folder_column"] = sampled_mass_df["image file path"].apply(lambda x: str(x)[:str(x).index("/")])
sampled_mass_df

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column
106,P_00108,mass,0,Mass-Training_P_00108_LEFT_MLO/raw.dcm,Mass-Training_P_00108_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00108_LEFT_MLO_1/roi1.dcm,Mass-Training_P_00108_LEFT_MLO
56,P_00065,mass,0,Mass-Training_P_00065_LEFT_CC/raw.dcm,Mass-Training_P_00065_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00065_LEFT_CC_1/roi1.dcm,Mass-Training_P_00065_LEFT_CC
432,P_00457,mass,0,Mass-Test_P_00457_LEFT_MLO/raw.dcm,Mass-Test_P_00457_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_00457_LEFT_MLO_1/roi1.dcm,Mass-Test_P_00457_LEFT_MLO
46,P_00055,mass,0,Mass-Training_P_00055_LEFT_CC/raw.dcm,Mass-Training_P_00055_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00055_LEFT_CC_1/roi1.dcm,Mass-Training_P_00055_LEFT_CC
519,P_00922,mass,0,Mass-Test_P_00922_RIGHT_MLO/raw.dcm,Mass-Test_P_00922_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_00922_RIGHT_MLO_1/roi1.dcm,Mass-Test_P_00922_RIGHT_MLO
...,...,...,...,...,...,...,...
108,P_00110,mass,5,Mass-Training_P_00110_LEFT_MLO/raw.dcm,Mass-Training_P_00110_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00110_LEFT_MLO_1/roi1.dcm,Mass-Training_P_00110_LEFT_MLO
577,P_01204,mass,5,Mass-Test_P_01204_RIGHT_MLO/raw.dcm,Mass-Test_P_01204_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_01204_RIGHT_MLO_1/roi1.dcm,Mass-Test_P_01204_RIGHT_MLO
545,P_00987,mass,5,Mass-Test_P_00987_LEFT_CC/raw.dcm,Mass-Test_P_00987_LEFT_CC_1/cropped1.dcm,Mass-Test_P_00987_LEFT_CC_1/roi1.dcm,Mass-Test_P_00987_LEFT_CC
565,P_01119,mass,5,Mass-Test_P_01119_LEFT_MLO/raw.dcm,Mass-Test_P_01119_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_01119_LEFT_MLO_1/roi1.dcm,Mass-Test_P_01119_LEFT_MLO


In [73]:
sampled_mass_df["roi_cropped_folder_column"] = sampled_mass_df["cropped image file path"].apply(lambda x: str(x)[:str(x).index("/")])
sampled_mass_df

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column,roi_cropped_folder_column
106,P_00108,mass,0,Mass-Training_P_00108_LEFT_MLO/raw.dcm,Mass-Training_P_00108_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00108_LEFT_MLO_1/roi1.dcm,Mass-Training_P_00108_LEFT_MLO,Mass-Training_P_00108_LEFT_MLO_1
56,P_00065,mass,0,Mass-Training_P_00065_LEFT_CC/raw.dcm,Mass-Training_P_00065_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00065_LEFT_CC_1/roi1.dcm,Mass-Training_P_00065_LEFT_CC,Mass-Training_P_00065_LEFT_CC_1
432,P_00457,mass,0,Mass-Test_P_00457_LEFT_MLO/raw.dcm,Mass-Test_P_00457_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_00457_LEFT_MLO_1/roi1.dcm,Mass-Test_P_00457_LEFT_MLO,Mass-Test_P_00457_LEFT_MLO_1
46,P_00055,mass,0,Mass-Training_P_00055_LEFT_CC/raw.dcm,Mass-Training_P_00055_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00055_LEFT_CC_1/roi1.dcm,Mass-Training_P_00055_LEFT_CC,Mass-Training_P_00055_LEFT_CC_1
519,P_00922,mass,0,Mass-Test_P_00922_RIGHT_MLO/raw.dcm,Mass-Test_P_00922_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_00922_RIGHT_MLO_1/roi1.dcm,Mass-Test_P_00922_RIGHT_MLO,Mass-Test_P_00922_RIGHT_MLO_1
...,...,...,...,...,...,...,...,...
108,P_00110,mass,5,Mass-Training_P_00110_LEFT_MLO/raw.dcm,Mass-Training_P_00110_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00110_LEFT_MLO_1/roi1.dcm,Mass-Training_P_00110_LEFT_MLO,Mass-Training_P_00110_LEFT_MLO_1
577,P_01204,mass,5,Mass-Test_P_01204_RIGHT_MLO/raw.dcm,Mass-Test_P_01204_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_01204_RIGHT_MLO_1/roi1.dcm,Mass-Test_P_01204_RIGHT_MLO,Mass-Test_P_01204_RIGHT_MLO_1
545,P_00987,mass,5,Mass-Test_P_00987_LEFT_CC/raw.dcm,Mass-Test_P_00987_LEFT_CC_1/cropped1.dcm,Mass-Test_P_00987_LEFT_CC_1/roi1.dcm,Mass-Test_P_00987_LEFT_CC,Mass-Test_P_00987_LEFT_CC_1
565,P_01119,mass,5,Mass-Test_P_01119_LEFT_MLO/raw.dcm,Mass-Test_P_01119_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_01119_LEFT_MLO_1/roi1.dcm,Mass-Test_P_01119_LEFT_MLO,Mass-Test_P_01119_LEFT_MLO_1


In [74]:
# import os
# import shutil

# source_directory = "../data CBIS-DDSM/CBIS-DDSM/"

# # Specify the destination directory where you want to copy the folders
# destination_directory = "../yolo_deneme_data/"

# # Create the destination directory if it doesn't exist
# os.makedirs(destination_directory, exist_ok=True)

# # Iterate over each folder name in the dataframe
# for folder_name in sampled_mass_df['roi_cropped_folder_column']:
#     # Construct the source and destination paths
#     source_path = os.path.join(source_directory, folder_name)
#     destination_path = os.path.join(destination_directory, folder_name)
    
#     try:
#         # Copy the folder to the destination directory
#         shutil.copytree(source_path, destination_path)
#         # print(f"Folder '{folder_name}' copied successfully.")
#     except Exception as e:
#         print(f"Error copying folder '{folder_name}': {str(e)}")

In [75]:
sampled_calc_df["raw_folder_column"] = sampled_calc_df["image file path"].apply(lambda x: str(x)[:str(x).index("/")])
sampled_calc_df["roi_cropped_folder_column"] = sampled_calc_df["cropped image file path"].apply(lambda x: str(x)[:str(x).index("/")])
sampled_calc_df

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column,roi_cropped_folder_column
603,P_00624,calcification,0,Calc-Training_P_00624_RIGHT_MLO/raw.dcm,Calc-Training_P_00624_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_00624_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_00624_RIGHT_MLO,Calc-Training_P_00624_RIGHT_MLO_1
1784,P_02220,calcification,0,Calc-Training_P_02220_LEFT_MLO/raw.dcm,Calc-Training_P_02220_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02220_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02220_LEFT_MLO,Calc-Training_P_02220_LEFT_MLO_1
1771,P_02159,calcification,0,Calc-Test_P_02159_LEFT_CC/raw.dcm,Calc-Test_P_02159_LEFT_CC_1/cropped1.dcm,Calc-Test_P_02159_LEFT_CC_1/roi1.dcm,Calc-Test_P_02159_LEFT_CC,Calc-Test_P_02159_LEFT_CC_1
1865,P_02563,calcification,0,Calc-Training_P_02563_RIGHT_MLO/raw.dcm,Calc-Training_P_02563_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_02563_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_02563_RIGHT_MLO,Calc-Training_P_02563_RIGHT_MLO_1
1826,P_02419,calcification,0,Calc-Training_P_02419_LEFT_MLO/raw.dcm,Calc-Training_P_02419_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02419_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02419_LEFT_MLO,Calc-Training_P_02419_LEFT_MLO_1
...,...,...,...,...,...,...,...,...
1003,P_01067,calcification,5,Calc-Test_P_01067_LEFT_CC/raw.dcm,Calc-Test_P_01067_LEFT_CC_1/cropped1.dcm,Calc-Test_P_01067_LEFT_CC_1/roi1.dcm,Calc-Test_P_01067_LEFT_CC,Calc-Test_P_01067_LEFT_CC_1
1064,P_01136,calcification,5,Calc-Training_P_01136_RIGHT_CC/raw.dcm,Calc-Training_P_01136_RIGHT_CC_1/cropped1.dcm,Calc-Training_P_01136_RIGHT_CC_1/roi1.dcm,Calc-Training_P_01136_RIGHT_CC,Calc-Training_P_01136_RIGHT_CC_1
294,P_00339,calcification,5,Calc-Training_P_00339_LEFT_CC/raw.dcm,Calc-Training_P_00339_LEFT_CC_1/cropped1.dcm,Calc-Training_P_00339_LEFT_CC_1/roi1.dcm,Calc-Training_P_00339_LEFT_CC,Calc-Training_P_00339_LEFT_CC_1
1260,P_01346,calcification,5,Calc-Training_P_01346_LEFT_CC/raw.dcm,Calc-Training_P_01346_LEFT_CC_1/cropped1.dcm,Calc-Training_P_01346_LEFT_CC_1/roi1.dcm,Calc-Training_P_01346_LEFT_CC,Calc-Training_P_01346_LEFT_CC_1


In [77]:
# for folder_name in sampled_calc_df['raw_folder_column']:
#     # Construct the source and destination paths
#     source_path = os.path.join(source_directory, folder_name)
#     destination_path = os.path.join(destination_directory, folder_name)
    
#     try:
#         # Copy the folder to the destination directory
#         shutil.copytree(source_path, destination_path)
#         # print(f"Folder '{folder_name}' copied successfully.")
#     except Exception as e:
#         print(f"Error copying folder '{folder_name}': {str(e)}")

Error copying folder 'Calc-Training_P_01292_RIGHT_CC': [WinError 183] Cannot create a file when that file already exists: '../yolo_deneme_data/Calc-Training_P_01292_RIGHT_CC'
Error copying folder 'Calc-Training_P_01292_RIGHT_CC': [WinError 183] Cannot create a file when that file already exists: '../yolo_deneme_data/Calc-Training_P_01292_RIGHT_CC'
Error copying folder 'Calc-Training_P_01292_RIGHT_MLO': [WinError 183] Cannot create a file when that file already exists: '../yolo_deneme_data/Calc-Training_P_01292_RIGHT_MLO'


In [83]:
sampled_mass_df.to_csv("../yolo_deneme_data/sampled_mass_df.csv", index = False)

In [84]:
sampled_calc_df.to_csv("../yolo_deneme_data/sampled_calc_df.csv", index = False)

In [85]:
sampled_calc_df.head()

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column,roi_cropped_folder_column
603,P_00624,calcification,0,Calc-Training_P_00624_RIGHT_MLO/raw.dcm,Calc-Training_P_00624_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_00624_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_00624_RIGHT_MLO,Calc-Training_P_00624_RIGHT_MLO_1
1784,P_02220,calcification,0,Calc-Training_P_02220_LEFT_MLO/raw.dcm,Calc-Training_P_02220_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02220_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02220_LEFT_MLO,Calc-Training_P_02220_LEFT_MLO_1
1771,P_02159,calcification,0,Calc-Test_P_02159_LEFT_CC/raw.dcm,Calc-Test_P_02159_LEFT_CC_1/cropped1.dcm,Calc-Test_P_02159_LEFT_CC_1/roi1.dcm,Calc-Test_P_02159_LEFT_CC,Calc-Test_P_02159_LEFT_CC_1
1865,P_02563,calcification,0,Calc-Training_P_02563_RIGHT_MLO/raw.dcm,Calc-Training_P_02563_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_02563_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_02563_RIGHT_MLO,Calc-Training_P_02563_RIGHT_MLO_1
1826,P_02419,calcification,0,Calc-Training_P_02419_LEFT_MLO/raw.dcm,Calc-Training_P_02419_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02419_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02419_LEFT_MLO,Calc-Training_P_02419_LEFT_MLO_1


In [97]:
def find_roi_coordinates(pixel_array):
    # Pixel array'deki beyaz piksellerin konumlarını bul
    if(pixel_array is None):
        return None
    white_pixels = np.where(pixel_array == 255)

    # ROI'nin koordinatlarını hesapla
    x_min = np.min(white_pixels[1])
    x_max = np.max(white_pixels[1])
    y_min = np.min(white_pixels[0])
    y_max = np.max(white_pixels[0])

    # ROI'nin merkez koordinatlarını ve boyutlarını hesapla
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    print("ROI max ve min coordinatlar", " xmin: ", x_min, " xmax: ", x_max, " ymin: ", y_min, " ymax: ", y_max)
    print(pixel_array.shape)
    return [x_center, y_center, width, height]


In [91]:

sampled_calc_df["ROI coordinates"] = sampled_calc_df["ROI mask file path"].apply(lambda x: str(find_roi_coordinates(pydicom.dcmread("../yolo_deneme_data/"+x).pixel_array)))

ROI max ve min coordinatlar  xmin:  1331  xmax:  1631  ymin:  2682  ymax:  3006
(5206, 2491)
ROI max ve min coordinatlar  xmin:  70  xmax:  582  ymin:  3442  ymax:  3894
(5386, 2311)
ROI max ve min coordinatlar  xmin:  1182  xmax:  1522  ymin:  1524  ymax:  1864
(6511, 4216)
ROI max ve min coordinatlar  xmin:  2647  xmax:  3035  ymin:  1786  ymax:  2146
(5266, 3511)
ROI max ve min coordinatlar  xmin:  134  xmax:  678  ymin:  3394  ymax:  4458
(5386, 2581)
ROI max ve min coordinatlar  xmin:  70  xmax:  1842  ymin:  3467  ymax:  4491
(5221, 2896)
ROI max ve min coordinatlar  xmin:  1653  xmax:  1925  ymin:  2635  ymax:  2935
(5191, 2701)
ROI max ve min coordinatlar  xmin:  2352  xmax:  2932  ymin:  2899  ymax:  3359
(6271, 4216)
ROI max ve min coordinatlar  xmin:  26  xmax:  294  ymin:  2161  ymax:  2465
(5281, 2341)
ROI max ve min coordinatlar  xmin:  1581  xmax:  2053  ymin:  3664  ymax:  4072
(5251, 3421)
ROI max ve min coordinatlar  xmin:  1274  xmax:  1754  ymin:  2454  ymax:  2950


In [92]:
sampled_calc_df.head()

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column,roi_cropped_folder_column,ROI coordinates
603,P_00624,calcification,0,Calc-Training_P_00624_RIGHT_MLO/raw.dcm,Calc-Training_P_00624_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_00624_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_00624_RIGHT_MLO,Calc-Training_P_00624_RIGHT_MLO_1,"[1481.0, 2844.0, 300, 324]"
1784,P_02220,calcification,0,Calc-Training_P_02220_LEFT_MLO/raw.dcm,Calc-Training_P_02220_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02220_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02220_LEFT_MLO,Calc-Training_P_02220_LEFT_MLO_1,"[326.0, 3668.0, 512, 452]"
1771,P_02159,calcification,0,Calc-Test_P_02159_LEFT_CC/raw.dcm,Calc-Test_P_02159_LEFT_CC_1/cropped1.dcm,Calc-Test_P_02159_LEFT_CC_1/roi1.dcm,Calc-Test_P_02159_LEFT_CC,Calc-Test_P_02159_LEFT_CC_1,"[1352.0, 1694.0, 340, 340]"
1865,P_02563,calcification,0,Calc-Training_P_02563_RIGHT_MLO/raw.dcm,Calc-Training_P_02563_RIGHT_MLO_1/cropped1.dcm,Calc-Training_P_02563_RIGHT_MLO_1/roi1.dcm,Calc-Training_P_02563_RIGHT_MLO,Calc-Training_P_02563_RIGHT_MLO_1,"[2841.0, 1966.0, 388, 360]"
1826,P_02419,calcification,0,Calc-Training_P_02419_LEFT_MLO/raw.dcm,Calc-Training_P_02419_LEFT_MLO_1/cropped1.dcm,Calc-Training_P_02419_LEFT_MLO_1/roi1.dcm,Calc-Training_P_02419_LEFT_MLO,Calc-Training_P_02419_LEFT_MLO_1,"[406.0, 3926.0, 544, 1064]"


In [100]:
sampled_mass_df["ROI coordinates"] = sampled_mass_df["ROI mask file path"].apply(lambda x: str(find_roi_coordinates(pydicom.dcmread("../yolo_deneme_data/"+x).pixel_array)))

49
ROI max ve min coordinatlar  xmin:  806  xmax:  1012  ymin:  3331  ymax:  3488
(4686, 2990)
50
ROI max ve min coordinatlar  xmin:  1302  xmax:  1671  ymin:  2403  ymax:  2734
(5386, 3331)
51
ROI max ve min coordinatlar  xmin:  1180  xmax:  1519  ymin:  3714  ymax:  4089
(5386, 2956)
52
ROI max ve min coordinatlar  xmin:  952  xmax:  1349  ymin:  2566  ymax:  2998
(5386, 3046)
53
ROI max ve min coordinatlar  xmin:  1258  xmax:  1951  ymin:  1867  ymax:  2424
(4582, 2155)
54
ROI max ve min coordinatlar  xmin:  1185  xmax:  1482  ymin:  3103  ymax:  3354
(5386, 3706)
55
ROI max ve min coordinatlar  xmin:  1280  xmax:  1692  ymin:  2285  ymax:  2663
(4686, 2859)
56
ROI max ve min coordinatlar  xmin:  2109  xmax:  2403  ymin:  2799  ymax:  2996
(4530, 2925)
57
ROI max ve min coordinatlar  xmin:  969  xmax:  1662  ymin:  1265  ymax:  2160
(4373, 1854)
58
ROI max ve min coordinatlar  xmin:  37  xmax:  212  ymin:  1466  ymax:  1673
(4543, 2951)
59
ROI max ve min coordinatlar  xmin:  2143  x

In [102]:
sampled_mass_df.head()

,patient_id,abnormality type,assessment,image file path,cropped image file path,ROI mask file path,raw_folder_column,roi_cropped_folder_column,ROI coordinates
106,P_00108,mass,0,Mass-Training_P_00108_LEFT_MLO/raw.dcm,Mass-Training_P_00108_LEFT_MLO_1/cropped1.dcm,Mass-Training_P_00108_LEFT_MLO_1/roi1.dcm,Mass-Training_P_00108_LEFT_MLO,Mass-Training_P_00108_LEFT_MLO_1,"[909.0, 3409.5, 206, 157]"
56,P_00065,mass,0,Mass-Training_P_00065_LEFT_CC/raw.dcm,Mass-Training_P_00065_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00065_LEFT_CC_1/roi1.dcm,Mass-Training_P_00065_LEFT_CC,Mass-Training_P_00065_LEFT_CC_1,"[1486.5, 2568.5, 369, 331]"
432,P_00457,mass,0,Mass-Test_P_00457_LEFT_MLO/raw.dcm,Mass-Test_P_00457_LEFT_MLO_1/cropped1.dcm,Mass-Test_P_00457_LEFT_MLO_1/roi1.dcm,Mass-Test_P_00457_LEFT_MLO,Mass-Test_P_00457_LEFT_MLO_1,"[1349.5, 3901.5, 339, 375]"
46,P_00055,mass,0,Mass-Training_P_00055_LEFT_CC/raw.dcm,Mass-Training_P_00055_LEFT_CC_1/cropped1.dcm,Mass-Training_P_00055_LEFT_CC_1/roi1.dcm,Mass-Training_P_00055_LEFT_CC,Mass-Training_P_00055_LEFT_CC_1,"[1150.5, 2782.0, 397, 432]"
519,P_00922,mass,0,Mass-Test_P_00922_RIGHT_MLO/raw.dcm,Mass-Test_P_00922_RIGHT_MLO_1/cropped1.dcm,Mass-Test_P_00922_RIGHT_MLO_1/roi1.dcm,Mass-Test_P_00922_RIGHT_MLO,Mass-Test_P_00922_RIGHT_MLO_1,"[1604.5, 2145.5, 693, 557]"


In [103]:
sampled_mass_df.to_csv("../yolo_deneme_data/sampled_mass_df.csv", index = False)
sampled_calc_df.to_csv("../yolo_deneme_data/sampled_calc_df.csv", index = False)